# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
# your code here

In [3]:
orders = pd.read_csv('Orders.csv', index_col=[0])
orders.head()

,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

### Sub Problem 1:
How to aggregate the  `amount_spent` for unique customers?

In [4]:
# Aggregate the amount_spent for unique customers, sort it to get the customer which spent most..
customers = orders.groupby('CustomerID', as_index=False)['amount_spent'].agg(sum)
#customers = customers.sort_values(by='amount_spent', ascending=False)
customers.head()

,CustomerID,amount_spent
0,12346,77183.60
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
4,12350,334.40


### Sub Problem 2:
How to select customers whose aggregated `amount_spent` is in a given quantile range?

In [5]:
# 95% Quantile
q95 = np.quantile(customers['amount_spent'], 0.95, axis=0)

# 75% Quantile
q75 = np.quantile(customers['amount_spent'], 0.75, axis=0)

# Customers which amount spent is over the 95% quantile
vips = customers[customers['amount_spent'] > q95]
display(vips.head())

# Customers which amount spent is in the range of the 75% and the 95% quantile
preferred = customers[(customers['amount_spent'] < q95) & (customers['amount_spent'] > q75)]
display(preferred.head())


,CustomerID,amount_spent
0,12346,77183.60
10,12357,6207.67
12,12359,6372.58
50,12409,11072.67
55,12415,124914.53


,CustomerID,amount_spent
1,12347,4310.00
2,12348,1797.24
3,12349,1757.55
5,12352,2506.04
9,12356,2811.43


### Sub Problem 3:
How to label selected customers as "VIP" or "Preferred"?

In [6]:
def label_customer(amount_spent):
    if amount_spent > q95:
        return "VIP"
    elif amount_spent < q95 and amount_spent > q75:
        return "Preferred"
    else:
        return np.nan

orders['role'] = customers['amount_spent'].apply(label_customer)
#orders = pd.concat([orders, roles], axis=1)

orders.head()


,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,role
0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,NaN


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [7]:
# your code here
x = orders.groupby(['Country','role']).agg({'role':'count'}).rename(columns={'role': 'num_roles'})
x = x.reset_index().sort_values(by='num_roles', ascending=False)

vips = x[x['role'] == 'VIP']
vips = vips[vips['num_roles'] == vips['num_roles'].max()]
vips


,Country,role,num_roles
10,United Kingdom,VIP,156


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [8]:
# your code here

prefs = x[x['role'] == 'Preferred']
prefs = prefs[prefs['num_roles'] == prefs['num_roles'].max()]
prefs


,Country,role,num_roles
9,United Kingdom,Preferred,604


The country with the most VIP+Preferred customers combined is UK with (604+156) 760 customers.